# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))

print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme pendant l' automne 

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [3]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    lines = [line for line in source_text.split('\n')]
    source_id_text = [[source_vocab_to_int[word] for word in line.split()] for line in lines]

    lines = [line for line in target_text.split('\n')]
    target_ids = [[target_vocab_to_int[word] for word in line.split()] for line in lines]

    target_ids = [line + [target_vocab_to_int['<EOS>']] for line in target_ids]
    
    return source_id_text, target_ids

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0
Default GPU Device: /gpu:0


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [7]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    # TODO: Implement Function
    inputs = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    keep_probability = tf.placeholder(tf.float32, name='keep_prob')
    target_sequence = tf.placeholder(tf.int32, [None], name='target_sequence_length')
    max_target = tf.reduce_max(target_sequence, name='max_target_len')
    source_sequence = tf.placeholder(tf.int32, name='source_sequence_length')
    
    return (inputs, targets, learning_rate, keep_probability, target_sequence, max_target, source_sequence)



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [8]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    target_seq = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    
    decoder_input = tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']), target_seq], 1, name='decoder_input')
    
    return decoder_input

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [9]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # TODO: Implement Function    
    embedding = tf.contrib.layers.embed_sequence (rnn_inputs, source_vocab_size, encoding_embedding_size)
    
    # RNN cell
    def make_cell(rnn_size):
        cell_encoded = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        return cell_encoded

    cell_encoded = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])

    output_encoded, state_encoded = tf.nn.dynamic_rnn(cell_encoded, embedding, dtype=tf.float32)
                                         #sequence_length=source_sequence_length, dtype=tf.float32)  # only state, drop output result
    
    return output_encoded, state_encoded

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [10]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    trainHelper =  tf.contrib.seq2seq.TrainingHelper (inputs=dec_embed_input, sequence_length=target_sequence_length, 
                                                     time_major=False, name = 'decoder_train_helper')
    
    basicDecoder =  tf.contrib.seq2seq.BasicDecoder (dec_cell, trainHelper, encoder_state, output_layer)
    
    dec_output, _ =  tf.contrib.seq2seq.dynamic_decode (basicDecoder, #output_time_major=False, 
                                                                    impute_finished=True,
                                                                    maximum_iterations=max_summary_length)
    # apply drop out here? perhaps no need for this example!
    #dec_output = tf.nn.dropout(dec_output, keep_prob=keep_prob)

    
    return dec_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [11]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function
    
    # good insights from https://medium.com/towards-data-science/text-summarization-with-amazon-reviews-41801c2210b
    start_tokens = tf.tile(tf.constant([start_of_sequence_id], dtype=tf.int32),  
                                       [batch_size], name='inference_start_tokens')
    
    inferenceEmbedHelper =  tf.contrib.seq2seq.GreedyEmbeddingHelper (dec_embeddings, start_tokens, end_of_sequence_id)
    
    inferenceDecoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, inferenceEmbedHelper, encoder_state, output_layer)
    
    
    inference_output, _ =  tf.contrib.seq2seq.dynamic_decode (inferenceDecoder, #output_time_major=False, 
                                                                    impute_finished=True,
                                                                    maximum_iterations=max_target_sequence_length)

    return inference_output



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [12]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input
    :param encoder_state: Encoder state
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    
    output_layer = Dense(target_vocab_size,
           kernel_initializer = tf.truncated_normal_initializer (mean=0.0, stddev=0.1), name='output_layer_decode')
    
    ############## Decoder cell: LSTM
    def make_cell(rnn_size):
        dec_cell = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        return dec_cell

    cell_decoded = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
    #cell_decoded = tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.BasicLSTMCell(rnn_size) for _ in range(num_layers)]),output_keep_prob=keep_prob)
    #############
    
    with tf.variable_scope("decode"):
        training_logits = decoding_layer_train(encoder_state, cell_decoded, dec_embed_input, 
                         target_sequence_length, max_target_sequence_length, 
                         output_layer, keep_prob)
        
    with tf.variable_scope("decode", reuse=True):
        start_tokens = target_vocab_to_int['<GO>'] #tf.tile(tf.constant([target_vocab_to_int['<GO>']], dtype=tf.int32), [batch_size], name='start_tokens')

        inference_logits = decoding_layer_infer(encoder_state, cell_decoded, dec_embeddings, start_tokens, 
                             target_vocab_to_int['<EOS>'], max_target_sequence_length,
                             target_vocab_size, output_layer, batch_size, keep_prob)
        
    return training_logits, inference_logits



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:

- Apply embedding to the input data for the encoder.
- Encode the input using your `encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob,  source_sequence_length, source_vocab_size, encoding_embedding_size)`.
- Process target data using your `process_decoder_input(target_data, target_vocab_to_int, batch_size)` function.
- Apply embedding to the target data for the decoder.
- Decode the encoded input using your `decoding_layer(dec_input, enc_state, target_sequence_length, max_target_sentence_length, rnn_size, num_layers, target_vocab_to_int, target_vocab_size, batch_size, keep_prob, dec_embedding_size)` function.

In [13]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    
    rnn_data_inputs = input_data 

    _, encoder_ini_state = encoding_layer(rnn_data_inputs, rnn_size, num_layers, keep_prob, source_sequence_length,
                                      source_vocab_size, enc_embedding_size)
    
    dec_input = process_decoder_input(target_data, target_vocab_to_int, batch_size)
        
    
    decode_train_logits, inference_logits = decoding_layer(dec_input, encoder_ini_state, 
                                                           target_sequence_length, max_target_sentence_length,
                                                           rnn_size, num_layers, 
                                                           target_vocab_to_int, target_vocab_size, batch_size,
                                                           keep_prob, dec_embedding_size)


    return decode_train_logits, inference_logits


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [14]:
# Number of Epochs
epochs = 10
# Batch Size
batch_size = 512
# RNN Size
rnn_size = 256
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 32
decoding_embedding_size = 32
# Learning Rate
learning_rate = 0.004
# Dropout Keep Probability
keep_probability = 0.5
display_step = 2

### Build the Graph
Build the graph using the neural network you implemented.

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    2/269 - Train Accuracy: 0.2994, Validation Accuracy: 0.3411, Loss: 4.8186
Epoch   0 Batch    4/269 - Train Accuracy: 0.2697, Validation Accuracy: 0.3427, Loss: 4.0004
Epoch   0 Batch    6/269 - Train Accuracy: 0.3129, Validation Accuracy: 0.3420, Loss: 3.4755
Epoch   0 Batch    8/269 - Train Accuracy: 0.3076, Validation Accuracy: 0.3695, Loss: 3.4253
Epoch   0 Batch   10/269 - Train Accuracy: 0.3097, Validation Accuracy: 0.3759, Loss: 3.3009
Epoch   0 Batch   12/269 - Train Accuracy: 0.3357, Validation Accuracy: 0.3920, Loss: 3.1845
Epoch   0 Batch   14/269 - Train Accuracy: 0.3513, Validation Accuracy: 0.3865, Loss: 2.9762
Epoch   0 Batch   16/269 - Train Accuracy: 0.3675, Validation Accuracy: 0.3957, Loss: 2.8990
Epoch   0 Batch   18/269 - Train Accuracy: 0.3490, Validation Accuracy: 0.4119, Loss: 2.8930
Epoch   0 Batch   20/269 - Train Accuracy: 0.3659, Validation Accuracy: 0.4226, Loss: 2.8218
Epoch   0 Batch   22/269 - Train Accuracy: 0.4069, Validation Accuracy

Epoch   0 Batch  180/269 - Train Accuracy: 0.5466, Validation Accuracy: 0.5525, Loss: 0.8390
Epoch   0 Batch  182/269 - Train Accuracy: 0.5396, Validation Accuracy: 0.5524, Loss: 0.8509
Epoch   0 Batch  184/269 - Train Accuracy: 0.5342, Validation Accuracy: 0.5634, Loss: 0.8708
Epoch   0 Batch  186/269 - Train Accuracy: 0.5182, Validation Accuracy: 0.5606, Loss: 0.8577
Epoch   0 Batch  188/269 - Train Accuracy: 0.5655, Validation Accuracy: 0.5574, Loss: 0.8039
Epoch   0 Batch  190/269 - Train Accuracy: 0.5485, Validation Accuracy: 0.5558, Loss: 0.7955
Epoch   0 Batch  192/269 - Train Accuracy: 0.5552, Validation Accuracy: 0.5711, Loss: 0.8121
Epoch   0 Batch  194/269 - Train Accuracy: 0.5560, Validation Accuracy: 0.5426, Loss: 0.8000
Epoch   0 Batch  196/269 - Train Accuracy: 0.5403, Validation Accuracy: 0.5657, Loss: 0.7881
Epoch   0 Batch  198/269 - Train Accuracy: 0.5352, Validation Accuracy: 0.5661, Loss: 0.8346
Epoch   0 Batch  200/269 - Train Accuracy: 0.5570, Validation Accuracy

Epoch   1 Batch   92/269 - Train Accuracy: 0.6376, Validation Accuracy: 0.6524, Loss: 0.5204
Epoch   1 Batch   94/269 - Train Accuracy: 0.6468, Validation Accuracy: 0.6433, Loss: 0.5185
Epoch   1 Batch   96/269 - Train Accuracy: 0.6410, Validation Accuracy: 0.6453, Loss: 0.5162
Epoch   1 Batch   98/269 - Train Accuracy: 0.6491, Validation Accuracy: 0.6547, Loss: 0.5129
Epoch   1 Batch  100/269 - Train Accuracy: 0.6580, Validation Accuracy: 0.6483, Loss: 0.4985
Epoch   1 Batch  102/269 - Train Accuracy: 0.6466, Validation Accuracy: 0.6531, Loss: 0.5026
Epoch   1 Batch  104/269 - Train Accuracy: 0.6091, Validation Accuracy: 0.6228, Loss: 0.5031
Epoch   1 Batch  106/269 - Train Accuracy: 0.6175, Validation Accuracy: 0.6371, Loss: 0.4968
Epoch   1 Batch  108/269 - Train Accuracy: 0.6442, Validation Accuracy: 0.6510, Loss: 0.4985
Epoch   1 Batch  110/269 - Train Accuracy: 0.6451, Validation Accuracy: 0.6578, Loss: 0.4858
Epoch   1 Batch  112/269 - Train Accuracy: 0.6443, Validation Accuracy

Epoch   2 Batch    4/269 - Train Accuracy: 0.6811, Validation Accuracy: 0.7002, Loss: 0.3422
Epoch   2 Batch    6/269 - Train Accuracy: 0.6622, Validation Accuracy: 0.6527, Loss: 0.3626
Epoch   2 Batch    8/269 - Train Accuracy: 0.6790, Validation Accuracy: 0.7128, Loss: 0.3868
Epoch   2 Batch   10/269 - Train Accuracy: 0.7073, Validation Accuracy: 0.7243, Loss: 0.3699
Epoch   2 Batch   12/269 - Train Accuracy: 0.6619, Validation Accuracy: 0.7077, Loss: 0.3584
Epoch   2 Batch   14/269 - Train Accuracy: 0.6874, Validation Accuracy: 0.7189, Loss: 0.3365
Epoch   2 Batch   16/269 - Train Accuracy: 0.7140, Validation Accuracy: 0.7224, Loss: 0.3287
Epoch   2 Batch   18/269 - Train Accuracy: 0.6958, Validation Accuracy: 0.7083, Loss: 0.3573
Epoch   2 Batch   20/269 - Train Accuracy: 0.7072, Validation Accuracy: 0.7236, Loss: 0.3416
Epoch   2 Batch   22/269 - Train Accuracy: 0.7367, Validation Accuracy: 0.7390, Loss: 0.3116
Epoch   2 Batch   24/269 - Train Accuracy: 0.7099, Validation Accuracy

Epoch   2 Batch  182/269 - Train Accuracy: 0.8171, Validation Accuracy: 0.8169, Loss: 0.1610
Epoch   2 Batch  184/269 - Train Accuracy: 0.8438, Validation Accuracy: 0.8303, Loss: 0.1627
Epoch   2 Batch  186/269 - Train Accuracy: 0.8430, Validation Accuracy: 0.8415, Loss: 0.1542
Epoch   2 Batch  188/269 - Train Accuracy: 0.8609, Validation Accuracy: 0.8410, Loss: 0.1510
Epoch   2 Batch  190/269 - Train Accuracy: 0.8560, Validation Accuracy: 0.8366, Loss: 0.1490
Epoch   2 Batch  192/269 - Train Accuracy: 0.8643, Validation Accuracy: 0.8463, Loss: 0.1441
Epoch   2 Batch  194/269 - Train Accuracy: 0.8503, Validation Accuracy: 0.8453, Loss: 0.1483
Epoch   2 Batch  196/269 - Train Accuracy: 0.8346, Validation Accuracy: 0.8479, Loss: 0.1413
Epoch   2 Batch  198/269 - Train Accuracy: 0.8449, Validation Accuracy: 0.8339, Loss: 0.1513
Epoch   2 Batch  200/269 - Train Accuracy: 0.8598, Validation Accuracy: 0.8482, Loss: 0.1445
Epoch   2 Batch  202/269 - Train Accuracy: 0.8608, Validation Accuracy

...

Epoch   6 Batch  186/269 - Train Accuracy: 0.9837, Validation Accuracy: 0.9757, Loss: 0.0122
Epoch   6 Batch  188/269 - Train Accuracy: 0.9778, Validation Accuracy: 0.9690, Loss: 0.0140
Epoch   6 Batch  190/269 - Train Accuracy: 0.9713, Validation Accuracy: 0.9767, Loss: 0.0145
Epoch   6 Batch  192/269 - Train Accuracy: 0.9860, Validation Accuracy: 0.9656, Loss: 0.0121
Epoch   6 Batch  194/269 - Train Accuracy: 0.9762, Validation Accuracy: 0.9678, Loss: 0.0148
Epoch   6 Batch  196/269 - Train Accuracy: 0.9768, Validation Accuracy: 0.9707, Loss: 0.0141
Epoch   6 Batch  198/269 - Train Accuracy: 0.9746, Validation Accuracy: 0.9743, Loss: 0.0157
Epoch   6 Batch  200/269 - Train Accuracy: 0.9863, Validation Accuracy: 0.9691, Loss: 0.0106
Epoch   6 Batch  202/269 - Train Accuracy: 0.9847, Validation Accuracy: 0.9662, Loss: 0.0147
Epoch   6 Batch  204/269 - Train Accuracy: 0.9782, Validation Accuracy: 0.9664, Loss: 0.0131
Epoch   6 Batch  206/269 - Train Accuracy: 0.9693, Validation Accuracy

Epoch   7 Batch   98/269 - Train Accuracy: 0.9737, Validation Accuracy: 0.9675, Loss: 0.0131
Epoch   7 Batch  100/269 - Train Accuracy: 0.9747, Validation Accuracy: 0.9735, Loss: 0.0145
Epoch   7 Batch  102/269 - Train Accuracy: 0.9788, Validation Accuracy: 0.9711, Loss: 0.0128
Epoch   7 Batch  104/269 - Train Accuracy: 0.9797, Validation Accuracy: 0.9730, Loss: 0.0132
Epoch   7 Batch  106/269 - Train Accuracy: 0.9798, Validation Accuracy: 0.9750, Loss: 0.0103
Epoch   7 Batch  108/269 - Train Accuracy: 0.9887, Validation Accuracy: 0.9768, Loss: 0.0121
Epoch   7 Batch  110/269 - Train Accuracy: 0.9788, Validation Accuracy: 0.9714, Loss: 0.0128
Epoch   7 Batch  112/269 - Train Accuracy: 0.9765, Validation Accuracy: 0.9689, Loss: 0.0152
Epoch   7 Batch  114/269 - Train Accuracy: 0.9824, Validation Accuracy: 0.9639, Loss: 0.0123
Epoch   7 Batch  116/269 - Train Accuracy: 0.9879, Validation Accuracy: 0.9704, Loss: 0.0111
Epoch   7 Batch  118/269 - Train Accuracy: 0.9755, Validation Accuracy

Epoch   8 Batch   10/269 - Train Accuracy: 0.9905, Validation Accuracy: 0.9768, Loss: 0.0098
Epoch   8 Batch   12/269 - Train Accuracy: 0.9798, Validation Accuracy: 0.9703, Loss: 0.0139
Epoch   8 Batch   14/269 - Train Accuracy: 0.9781, Validation Accuracy: 0.9695, Loss: 0.0106
Epoch   8 Batch   16/269 - Train Accuracy: 0.9761, Validation Accuracy: 0.9641, Loss: 0.0132
Epoch   8 Batch   18/269 - Train Accuracy: 0.9757, Validation Accuracy: 0.9633, Loss: 0.0121
Epoch   8 Batch   20/269 - Train Accuracy: 0.9849, Validation Accuracy: 0.9660, Loss: 0.0114
Epoch   8 Batch   22/269 - Train Accuracy: 0.9841, Validation Accuracy: 0.9714, Loss: 0.0107
Epoch   8 Batch   24/269 - Train Accuracy: 0.9823, Validation Accuracy: 0.9711, Loss: 0.0100
Epoch   8 Batch   26/269 - Train Accuracy: 0.9761, Validation Accuracy: 0.9663, Loss: 0.0104
Epoch   8 Batch   28/269 - Train Accuracy: 0.9784, Validation Accuracy: 0.9600, Loss: 0.0122
Epoch   8 Batch   30/269 - Train Accuracy: 0.9810, Validation Accuracy

Epoch   8 Batch  188/269 - Train Accuracy: 0.9823, Validation Accuracy: 0.9792, Loss: 0.0110
Epoch   8 Batch  190/269 - Train Accuracy: 0.9862, Validation Accuracy: 0.9778, Loss: 0.0103
Epoch   8 Batch  192/269 - Train Accuracy: 0.9887, Validation Accuracy: 0.9711, Loss: 0.0078
Epoch   8 Batch  194/269 - Train Accuracy: 0.9891, Validation Accuracy: 0.9708, Loss: 0.0102
Epoch   8 Batch  196/269 - Train Accuracy: 0.9818, Validation Accuracy: 0.9762, Loss: 0.0111
Epoch   8 Batch  198/269 - Train Accuracy: 0.9851, Validation Accuracy: 0.9737, Loss: 0.0110
Epoch   8 Batch  200/269 - Train Accuracy: 0.9886, Validation Accuracy: 0.9697, Loss: 0.0088
Epoch   8 Batch  202/269 - Train Accuracy: 0.9826, Validation Accuracy: 0.9668, Loss: 0.0101
Epoch   8 Batch  204/269 - Train Accuracy: 0.9879, Validation Accuracy: 0.9711, Loss: 0.0094
Epoch   8 Batch  206/269 - Train Accuracy: 0.9850, Validation Accuracy: 0.9757, Loss: 0.0123
Epoch   8 Batch  208/269 - Train Accuracy: 0.9870, Validation Accuracy

Epoch   9 Batch  100/269 - Train Accuracy: 0.9785, Validation Accuracy: 0.9658, Loss: 0.0122
Epoch   9 Batch  102/269 - Train Accuracy: 0.9787, Validation Accuracy: 0.9719, Loss: 0.0107
Epoch   9 Batch  104/269 - Train Accuracy: 0.9873, Validation Accuracy: 0.9694, Loss: 0.0097
Epoch   9 Batch  106/269 - Train Accuracy: 0.9862, Validation Accuracy: 0.9688, Loss: 0.0091
Epoch   9 Batch  108/269 - Train Accuracy: 0.9893, Validation Accuracy: 0.9739, Loss: 0.0093
Epoch   9 Batch  110/269 - Train Accuracy: 0.9886, Validation Accuracy: 0.9678, Loss: 0.0080
Epoch   9 Batch  112/269 - Train Accuracy: 0.9808, Validation Accuracy: 0.9715, Loss: 0.0116
Epoch   9 Batch  114/269 - Train Accuracy: 0.9841, Validation Accuracy: 0.9707, Loss: 0.0093
Epoch   9 Batch  116/269 - Train Accuracy: 0.9897, Validation Accuracy: 0.9712, Loss: 0.0101
Epoch   9 Batch  118/269 - Train Accuracy: 0.9851, Validation Accuracy: 0.9648, Loss: 0.0097
Epoch   9 Batch  120/269 - Train Accuracy: 0.9881, Validation Accuracy

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [19]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [20]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    sentence_split = [word.lower() for word in sentence.split()]
    sentence_seq = [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in sentence_split]

    return sentence_seq


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [21]:
translate_sentence = 'he saw a old yellow truck .'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [186, 227, 33, 202, 187, 76, 152]
  English Words: ['he', 'saw', 'a', 'old', 'yellow', 'truck', '.']

Prediction
  Word Ids:      [151, 11, 25, 25, 192, 179, 0, 0, 0, 0, 0, 0, 0, 0]
  French Words: il a un un vieux camion <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.